Doron Schwartz, 05/22/23,Final Project

In [ ]:
!pip install scikit-surprise

import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.metrics.pairwise import cosine_similarity
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate, train_test_split
from surprise.accuracy import rmse
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
from sklearn.preprocessing import StandardScaler
from keras.layers import Input, Dense
from keras.models import Model



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


We load in the imbd data with the rating, and we also load in the genre data for the Simple and Hybrid Models later. We rename the columns for reference, with the last bunch of columns being the vector of genres

In [ ]:
df = pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u1.base", sep='\t', header=None, names=["userId", "movieId", "rating", "timestamp"])

item_data = pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u.item", sep='|', header=None, encoding="ISO-8859-1")

item_data.columns = ["movieId", "title", "release_date", "video_release_date", "IMDb_URL", "unknown", "Action", "Adventure", "Animation", "Children's", "Comedy", "Crime", "Documentary", "Drama", "Fantasy", "Film-Noir", "Horror", "Musical", "Mystery", "Romance", "Sci-Fi", "Thriller", "War", "Western"]

Do a train-test split.

In [ ]:
train_data, test_data = sklearn.model_selection.train_test_split(df, test_size=0.2, random_state=42)

Baseline Model

For the baseline model, we will just take the mean rating across all the ratings in the training data is calculated as the baseline rating. The baseline model predicts this mean rating for all cases in the test data.

We can take the mean of the training data

In [ ]:
mean_rating = train_data['rating'].mean()


We have a quick function that retrieves the mean rating, and we run it through the test data

In [ ]:
def baseline_model():
    return mean_rating

baseline_predictions = [baseline_model() for _ in range(len(test_data))]

Quick functions to get the RMSE

In [ ]:
def rmse(predictions, targets):
    return np.sqrt(((predictions - targets) ** 2).mean())


Get the Baseline RMSE and print it out

In [ ]:
baseline_rmse = rmse(baseline_predictions, test_data['rating'])
print("Baseline RMSE:", baseline_rmse)


Baseline RMSE: 1.1244493378879115


Simple Model

The simple model will be focused on collarbitive filtering, and will be based on a SVD model, which is a Singular Value Decompisiton(SVD), which will take the matrix and reduce it to the most important featues.

We have the rating and genre merged here for Hybrid model purposes, but join them now at the Model stage at MovieID

In [ ]:
merged_data = pd.merge(df, item_data, on="movieId", how="left")

Using the Surprise Library, we load in the Scaler which has the ratings between 1 and 5, and we load in the data into a Surprise Dataset

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[['userId', 'movieId', 'rating']], reader)


Do a Train/Test split on the processed data

In [ ]:
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)


Make the matrix factorization/SVD Model

In [ ]:
model_one = SVD()

Fit the model to the training data

In [ ]:
model_one.fit(trainset)

Pull predictions from the model, get the actual ratings and the predicted ratings for the model

In [ ]:
predictions = model_one.test(testset)
actual_ratings = np.array([pred.r_ui for pred in predictions])
predicted_ratings = np.array([pred.est for pred in predictions])


Calculate the RMSE from the predictions

In [ ]:
mse = mean_squared_error(actual_ratings, predicted_ratings)
rmse_score = np.sqrt(mse)
print("RMSE:", rmse_score)



RMSE: 0.9404862627972276


Hybrid Model

In the Hybrid Model, we will combine the Collorabtaive filtering of the SVD, and take the Genres and embedd them to get a content filtering aspect. WE can then throw it into the DNN and see how well it is able to predict at that point.

From before, we had the merged data, so we can just throw it into the Scaler and load into a Surprise Dataset

In [ ]:
# Normalize the rating scale using Surprise library
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(merged_data[['userId', 'movieId', 'rating']], reader)


Pull out the data we need from the sheet.

In [ ]:
df_data = pd.DataFrame(data.raw_ratings, columns=["userId", "movieId", "rating", "timestamp"])

Split the Data into train/test

In [ ]:
train_data, test_data = sklearn.model_selection.train_test_split(df_data, test_size=0.2, random_state=42)

Make the Surprise dataset from the training

In [ ]:
trainset = Dataset.load_from_df(train_data[['userId', 'movieId', 'rating']], reader)

Build the SVD model and get it fit for the trainset

In [ ]:
svd_model = SVD()
svd_model.fit(trainset.build_full_trainset())

Retrieve the Genre data

In [ ]:
genre_features = merged_data[['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']].values

Build the Standard Scaler, which scales the features.

In [ ]:
scaler = StandardScaler()
genre_features_scaled = scaler.fit_transform(genre_features)

No we want to build an autoencoder for the genre data, fit in the genre data and the encoding demnsions

In [ ]:
input_dim_genre = genre_features_scaled.shape[1]
encoding_dim_genre = 32

Make the encoding layer with the correct dimensions from above

In [ ]:
input_layer_genre = Input(shape=(input_dim_genre,))
encoder_genre = Dense(encoding_dim_genre, activation='relu')(input_layer_genre)
decoder_genre = Dense(input_dim_genre, activation='sigmoid')(encoder_genre)


Build the Autoencoder itself

In [ ]:
autoencoder_genre = Model(inputs=input_layer_genre, outputs=decoder_genre)
autoencoder_genre.compile(optimizer='adam', loss='mean_squared_error')
autoencoder_genre.fit(genre_features_scaled, genre_features_scaled, epochs=10, batch_size=32)

Epoch 1/10
2500/2500 [==============================] - 10s 4ms/step - loss: 0.6238
Epoch 2/10
2500/2500 [==============================] - 9s 4ms/step - loss: 0.5621
Epoch 3/10
2500/2500 [==============================] - 6s 2ms/step - loss: 0.5615
Epoch 4/10
2500/2500 [==============================] - 4s 2ms/step - loss: 0.5614
Epoch 5/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.5613
Epoch 6/10
2500/2500 [==============================] - 4s 2ms/step - loss: 0.5613
Epoch 7/10
2500/2500 [==============================] - 4s 2ms/step - loss: 0.5613
Epoch 8/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.5613
Epoch 9/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.5613
Epoch 10/10
2500/2500 [==============================] - 5s 2ms/step - loss: 0.5613


Retrieve the genre embeddings and the SVd embeddings

In [ ]:
genre_embeddings = autoencoder_genre.predict(genre_features_scaled)
svd_embeddings = svd_model.qi

2500/2500 [==============================] - 4s 2ms/step


Get the test ratings.

In [ ]:
test_ratings = test_data['rating'].values

Truncate the embeddings in order to have a working combination

In [ ]:
genre_embeddings_truncated = genre_embeddings[:svd_embeddings.shape[0]]

Combine the SVD and Genre embeddings

In [ ]:
combined_embeddings = np.concatenate((genre_embeddings_truncated, svd_embeddings), axis=1)

Now we can build the DNN Model, with three layers of 64/32 and the linear at the end.

In [ ]:
model_two = Sequential()
model_two.add(Dense(64, activation='relu', input_shape=(combined_embeddings.shape[1],)))
model_two.add(Dense(32, activation='relu'))
model_two.add(Dense(1, activation='linear'))

Compile and fit the DNN model

In [ ]:
model_two.compile(optimizer=Adam(), loss='mean_squared_error')
model_two.fit(combined_embeddings, test_ratings[:combined_embeddings.shape[0]], epochs=10, batch_size=32)

Epoch 1/10
51/51 [==============================] - 1s 2ms/step - loss: 7.0637
Epoch 2/10
51/51 [==============================] - 0s 2ms/step - loss: 1.6394
Epoch 3/10
51/51 [==============================] - 0s 3ms/step - loss: 1.3484
Epoch 4/10
51/51 [==============================] - 0s 2ms/step - loss: 1.2238
Epoch 5/10
51/51 [==============================] - 0s 2ms/step - loss: 1.1518
Epoch 6/10
51/51 [==============================] - 0s 2ms/step - loss: 1.0772
Epoch 7/10
51/51 [==============================] - 0s 2ms/step - loss: 1.0290
Epoch 8/10
51/51 [==============================] - 0s 2ms/step - loss: 0.9826
Epoch 9/10
51/51 [==============================] - 0s 2ms/step - loss: 0.9374
Epoch 10/10
51/51 [==============================] - 0s 2ms/step - loss: 0.9020


Get the predicited rating and pull

In [ ]:

predicted_ratings = model_two.predict(combined_embeddings)
rmse_score = np.sqrt(mean_squared_error(test_ratings[:combined_embeddings.shape[0]], predicted_ratings))
print("RMSE:", rmse_score)

51/51 [==============================] - 0s 2ms/step
RMSE: 0.9152785904274006


Testing

We will do seperate testing on the Simple and Hybrid models, we will load in the test file and process it like we did the training data

In [ ]:
df_test_data = pd.read_csv("http://files.grouplens.org/datasets/movielens/ml-100k/u1.test", sep='\t', header=None, names=["userId", "movieId", "rating", "timestamp"])
test_ratings = df_test_data['rating'].values

Simple

Push the test data through the same way we did in the train/test split, and pull the RMSE

In [ ]:
test_ratings = df_test_data['rating'].values

# Prediction
predicted_ratings_simple = model_one.test(df_test_data[['userId', 'movieId', 'rating']].values)

#Got the ratings
predicted_ratings_simple = [pred.est for pred in predicted_ratings_simple]

# Get RMSE
rmse_score_simple = np.sqrt(mean_squared_error(test_ratings, predicted_ratings_simple))
print("Simple Model (SVD) RMSE:", rmse_score_simple)

Simple Model (SVD) RMSE: 0.9595426452167031


Hybrid

We will process the test data to make available for prediction

Build the SVD embeddings for the test, and concatante with the Genre embedding

In [ ]:
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df_test_data[['userId', 'movieId', 'rating']], reader)
svd_model = SVD()
svd_model.fit(data.build_full_trainset())
svd_embeddings = svd_model.qi

Add in the genre to one vector

In [ ]:
genre_embeddings_truncated = genre_embeddings[:svd_embeddings.shape[0]]
combined_embeddings = np.concatenate((genre_embeddings_truncated, svd_embeddings), axis=1)

Pull the predictions

In [ ]:
predicted_ratings = model_two.predict(combined_embeddings)
rmse_score = np.sqrt(mean_squared_error(test_ratings[:combined_embeddings.shape[0]], predicted_ratings))
print("RMSE:", rmse_score)

45/45 [==============================] - 0s 2ms/step
RMSE: 1.3130233614865616
